# ESG Scoring using LLM Models
## Comprehensive comparison of IBM Granite vs Mistral Large

This notebook implements LLM-based ESG scoring following the exercise methodology and evaluates against benchmark scores from benchMark.md.

In [ ]:
# 1. Import Required Libraries
import pandas as pd
import numpy as np
import os
import json
from sklearn.metrics import classification_report, mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict
import warnings
warnings.filterwarnings('ignore')

# LLM-specific imports
from decouple import config
from dotenv import load_dotenv
import time
from tqdm import tqdm

print("🚀 LLM ESG Analysis Setup Complete")
print("=" * 50)

In [ ]:
# 2. Load and Prepare ESG Data

def load_benchmark_scores():
    """Load benchmark ESG scores from benchMark.md"""
    benchmark_scores = {
        'Nike': 70, 'Apple': 70, 'Boeing': 79, 'Disney': 68, 'Eli Lilly': 64, 'FedEx': 71,
        'Johnson & Johnson': 85, 'JPMorgan Chase': 80, 'McDonald\'s': 66, 'Meta': 60,
        'Microsoft': 87, 'Netflix': 37, 'NVIDIA': 77, 'Tesla': 72, 'Google': 81
    }
    return benchmark_scores

def load_company_documents():
    """Load all ESG-related documents for each company"""
    data_path = "data"
    company_texts = {}
    
    # Company directory mapping
    company_dirs = {
        'Nike': 'nike', 'Apple': 'apple', 'Boeing': 'boeing', 'Disney': 'disney',
        'Eli Lilly': 'elililly', 'FedEx': 'fedex', 'Johnson & Johnson': 'johnsonandjohnson',
        'JPMorgan Chase': 'jpmorganchase', 'McDonald\'s': 'mcdonald', 'Meta': 'meta',
        'Microsoft': 'microsoft', 'Netflix': 'netflix', 'NVIDIA': 'nvidia',
        'Tesla': 'tesla', 'Google': 'google'
    }
    
    print("📂 Loading company documents...")
    for company, dir_name in company_dirs.items():
        company_path = os.path.join(data_path, dir_name)
        if os.path.exists(company_path):
            all_text = ""
            file_count = 0
            
            # Load all relevant files
            for file in os.listdir(company_path):
                if file.endswith(('.md', '.txt')):
                    file_path = os.path.join(company_path, file)
                    try:
                        with open(file_path, 'r', encoding='utf-8') as f:
                            content = f.read()
                            # Truncate very long documents for LLM processing
                            if len(content) > 8000:  # LLM token limit consideration
                                content = content[:8000]
                            all_text += content + " "
                            file_count += 1
                    except Exception as e:
                        print(f"Error reading {file_path}: {e}")
            
            if all_text.strip():
                company_texts[company] = all_text.strip()
                print(f"✓ {company}: {file_count} files loaded")
            else:
                print(f"✗ {company}: No text found")
        else:
            print(f"✗ {company}: Directory not found")
    
    return company_texts

# Load data
benchmark_scores = load_benchmark_scores()
company_texts = load_company_documents()

# Create dataset
data = []
for company in company_texts.keys():
    if company in benchmark_scores:
        data.append({
            'company': company,
            'text': company_texts[company],
            'esg_score': benchmark_scores[company]
        })

esg_df = pd.DataFrame(data)

print(f"\n📊 Dataset: {len(esg_df)} companies with both text and scores")
print(esg_df[['company', 'esg_score']].to_string(index=False))

In [ ]:
# 3. Create ESG Score Categories

def create_esg_categories(df):
    """Create categorical labels from continuous ESG scores"""
    
    # Create quartile-based categories
    df['esg_quartile'] = pd.qcut(df['esg_score'], q=4, labels=['Low', 'Medium-Low', 'Medium-High', 'High'])
    
    # Create three-tier categorization for LLM classification
    df['esg_tier'] = pd.cut(df['esg_score'], 
                           bins=[0, 50, 75, 100], 
                           labels=['Poor', 'Good', 'Excellent'],
                           include_lowest=True)
    
    # Create binary classification (above/below median)
    median_score = df['esg_score'].median()
    df['esg_binary'] = df['esg_score'].apply(lambda x: 'High' if x >= median_score else 'Low')
    
    print("📈 ESG Score Distribution:")
    print(f"Range: {df['esg_score'].min()} - {df['esg_score'].max()}")
    print(f"Mean: {df['esg_score'].mean():.1f}")
    print(f"Median: {median_score:.1f}")
    
    return df

# Apply categorization
esg_df = create_esg_categories(esg_df)

In [ ]:
# 4. Setup LLM Clients

def setup_llm_clients():
    """Setup IBM Watsonx LLM clients for both models"""
    
    try:
        # Load environment variables
        env_path = "/Users/wenlong/Documents/GitHub/ma2/assignments/.env"
        if os.path.exists(env_path):
            load_dotenv(dotenv_path=env_path)
        
        WX_API_KEY = os.getenv("WX_API_KEY")
        
        if not WX_API_KEY:
            raise ValueError("WX_API_KEY not found in environment variables")
        
        # Import IBM Watsonx AI modules
        from ibm_watsonx_ai import APIClient, Credentials
        from ibm_watsonx_ai.foundation_models import ModelInference
        from ibm_watsonx_ai.foundation_models.schema import TextGenParameters
        
        # Setup credentials
        credentials = Credentials(
            url="https://us-south.ml.cloud.ibm.com",
            api_key=WX_API_KEY
        )
        
        # Create client
        client = APIClient(
            credentials=credentials, 
            project_id="fb3ce137-1a1e-411b-b5f4-d66b00a139f0"
        )
        
        # Setup IBM Granite model
        granite_params = TextGenParameters(
            temperature=0,
            max_new_tokens=50,
            min_new_tokens=1,
            stop_sequences=[".", "\n"],
        )
        
        granite_model = ModelInference(
            api_client=client,
            model_id="ibm/granite-13b-instruct-v2",
            params=granite_params
        )
        
        # Setup Mistral Large model (fixed parameters)
        mistral_params = TextGenParameters(
            temperature=0,
            max_new_tokens=100,  # Increased for better responses
            min_new_tokens=1,
            # Removed stop_sequences to prevent empty responses
        )
        
        mistral_model = ModelInference(
            api_client=client,
            model_id="mistralai/mistral-large",
            params=mistral_params
        )
        
        print("✅ LLM clients setup successful")
        print("Models available:")
        print("- IBM Granite-13B-Instruct-v2")
        print("- Mistral Large")
        
        return client, granite_model, mistral_model
        
    except Exception as e:
        print(f"❌ Error setting up LLM clients: {e}")
        return None, None, None

# Setup clients
llm_client, granite_model, mistral_model = setup_llm_clients()

In [ ]:
# 5. Define LLM Prompts

# Classification prompt for tier prediction
classification_prompt_template = """Analyze the following company's ESG (Environmental, Social, Governance) information and classify their ESG performance.

Based on the provided information, classify the company's ESG performance into one of these categories:
- Poor: Significant ESG issues, limited sustainability efforts
- Good: Moderate ESG performance, some sustainability initiatives
- Excellent: Strong ESG leadership, comprehensive sustainability programs

Company Information:
{text}

ESG Classification:"""

# Binary classification prompt
binary_prompt_template = """Analyze the following company's ESG (Environmental, Social, Governance) information and provide a binary classification.

Based on the provided information, classify the company's ESG performance as:
- High: Above-average ESG performance
- Low: Below-average ESG performance

Company Information:
{text}

ESG Classification:"""

# Scoring prompt for numerical prediction
scoring_prompt_template = """Analyze the following company's ESG (Environmental, Social, Governance) information and provide a numerical score.

Based on the provided information, rate the company's ESG performance on a scale of 0-100, where:
- 0-30: Poor ESG performance
- 31-70: Good ESG performance  
- 71-100: Excellent ESG performance

Provide only the numerical score.

Company Information:
{text}

ESG Score:"""

print("📝 LLM prompts defined")
print("- Classification (3-tier)")
print("- Binary classification")
print("- Numerical scoring (0-100)")

In [ ]:
# 6. LLM Processing Functions

def llm_classify_companies(model, prompt_template, companies_data, task_name="Classification"):
    """Use LLM to classify companies"""
    predictions = {}
    
    print(f"🤖 {task_name} with LLM...")
    
    for _, row in tqdm(companies_data.iterrows(), total=len(companies_data), desc=f"LLM {task_name}"):
        company = row['company']
        text = row['text']
        
        try:
            # Format prompt
            prompt = prompt_template.format(text=text)
            
            # Get LLM response
            response = model.generate(prompt=prompt)
            
            # Extract prediction from response
            if 'results' in response and len(response['results']) > 0:
                prediction = response['results'][0]['generated_text'].strip()
                
                # Clean up prediction
                prediction = prediction.replace('\n', '').strip()
                
                # For scoring tasks, extract number
                if task_name == "Scoring":
                    try:
                        # Extract number from response
                        import re
                        numbers = re.findall(r'\d+', prediction)
                        if numbers:
                            prediction = int(numbers[0])
                            # Ensure score is within valid range
                            prediction = max(0, min(100, prediction))
                        else:
                            prediction = 50  # Default score if no number found
                    except:
                        prediction = 50
                        
                predictions[company] = prediction
            else:
                print(f"Warning: No response for {company}")
                predictions[company] = 'Good' if task_name != "Scoring" else 50
                
        except Exception as e:
            print(f"Error processing {company}: {e}")
            predictions[company] = 'Good' if task_name != "Scoring" else 50
            
        # Small delay to avoid rate limiting
        time.sleep(0.1)
    
    print(f"✅ {task_name} completed")
    return predictions

print("🔧 LLM processing functions ready")

In [ ]:
# 7. Run IBM Granite Model

if granite_model is not None:
    print("🚀 Running IBM Granite Model Analysis")
    print("=" * 50)
    
    # 1. Tier Classification
    granite_tier_predictions = llm_classify_companies(
        granite_model, classification_prompt_template, esg_df, "Tier Classification"
    )
    
    # 2. Binary Classification
    granite_binary_predictions = llm_classify_companies(
        granite_model, binary_prompt_template, esg_df, "Binary Classification"
    )
    
    # 3. Score Prediction
    granite_score_predictions = llm_classify_companies(
        granite_model, scoring_prompt_template, esg_df, "Scoring"
    )
    
    # Add predictions to dataframe
    esg_df['granite_tier_pred'] = esg_df['company'].map(granite_tier_predictions)
    esg_df['granite_binary_pred'] = esg_df['company'].map(granite_binary_predictions)
    esg_df['granite_score_pred'] = esg_df['company'].map(granite_score_predictions)
    
    print("\n✅ IBM Granite analysis complete")
    
else:
    print("❌ IBM Granite model not available")

In [ ]:
# 8. Run Mistral Large Model

if mistral_model is not None:
    print("🚀 Running Mistral Large Model Analysis")
    print("=" * 50)
    
    # 1. Tier Classification
    mistral_tier_predictions = llm_classify_companies(
        mistral_model, classification_prompt_template, esg_df, "Tier Classification"
    )
    
    # 2. Binary Classification
    mistral_binary_predictions = llm_classify_companies(
        mistral_model, binary_prompt_template, esg_df, "Binary Classification"
    )
    
    # 3. Score Prediction
    mistral_score_predictions = llm_classify_companies(
        mistral_model, scoring_prompt_template, esg_df, "Scoring"
    )
    
    # Add predictions to dataframe
    esg_df['mistral_tier_pred'] = esg_df['company'].map(mistral_tier_predictions)
    esg_df['mistral_binary_pred'] = esg_df['company'].map(mistral_binary_predictions)
    esg_df['mistral_score_pred'] = esg_df['company'].map(mistral_score_predictions)
    
    print("\n✅ Mistral Large analysis complete")
    
else:
    print("❌ Mistral Large model not available")

In [ ]:
# 9. Model Performance Evaluation

def evaluate_model_performance(df, model_prefix):
    """Evaluate model performance across all tasks"""
    
    results = {}
    
    # Score prediction evaluation (main metric)
    score_col = f'{model_prefix}_score_pred'
    if score_col in df.columns:
        true_scores = df['esg_score']
        pred_scores = df[score_col]
        
        mae = mean_absolute_error(true_scores, pred_scores)
        rmse = np.sqrt(mean_squared_error(true_scores, pred_scores))
        r2 = r2_score(true_scores, pred_scores)
        
        results['scoring'] = {
            'mae': mae,
            'rmse': rmse,
            'r2': r2
        }
        
        print(f"📊 {model_prefix.upper()} SCORING PERFORMANCE:")
        print(f"MAE: {mae:.2f}")
        print(f"RMSE: {rmse:.2f}")
        print(f"R²: {r2:.4f}")
    
    # Classification evaluation
    tier_col = f'{model_prefix}_tier_pred'
    binary_col = f'{model_prefix}_binary_pred'
    
    if tier_col in df.columns:
        tier_accuracy = (df['esg_tier'] == df[tier_col]).mean()
        results['tier_accuracy'] = tier_accuracy
        print(f"\nTier Classification Accuracy: {tier_accuracy:.3f}")
    
    if binary_col in df.columns:
        binary_accuracy = (df['esg_binary'] == df[binary_col]).mean()
        results['binary_accuracy'] = binary_accuracy
        print(f"Binary Classification Accuracy: {binary_accuracy:.3f}")
    
    return results

# Evaluate both models
if 'granite_score_pred' in esg_df.columns:
    granite_results = evaluate_model_performance(esg_df, 'granite')

if 'mistral_score_pred' in esg_df.columns:
    print("\n" + "-"*50)
    mistral_results = evaluate_model_performance(esg_df, 'mistral')

In [ ]:
# 10. Comprehensive Model Comparison & Visualization

if 'granite_score_pred' in esg_df.columns and 'mistral_score_pred' in esg_df.columns:
    
    print("\n" + "="*80)
    print("🏆 COMPREHENSIVE MODEL COMPARISON")
    print("="*80)
    
    # Create comparison dataframe
    comparison_df = esg_df[['company', 'esg_score', 'granite_score_pred', 'mistral_score_pred']].copy()
    comparison_df.columns = ['Company', 'Benchmark_Score', 'IBM_Granite', 'Mistral_Large']
    
    # Calculate performance metrics
    granite_mae = mean_absolute_error(comparison_df['Benchmark_Score'], comparison_df['IBM_Granite'])
    granite_rmse = np.sqrt(mean_squared_error(comparison_df['Benchmark_Score'], comparison_df['IBM_Granite']))
    granite_r2 = r2_score(comparison_df['Benchmark_Score'], comparison_df['IBM_Granite'])
    
    mistral_mae = mean_absolute_error(comparison_df['Benchmark_Score'], comparison_df['Mistral_Large'])
    mistral_rmse = np.sqrt(mean_squared_error(comparison_df['Benchmark_Score'], comparison_df['Mistral_Large']))
    mistral_r2 = r2_score(comparison_df['Benchmark_Score'], comparison_df['Mistral_Large'])
    
    # Performance summary
    print(f"\n📊 PERFORMANCE SUMMARY:")
    print(f"{'Model':<15} {'MAE':<10} {'RMSE':<10} {'R²':<10}")
    print("-" * 45)
    print(f"{'IBM Granite':<15} {granite_mae:<10.2f} {granite_rmse:<10.2f} {granite_r2:<10.4f}")
    print(f"{'Mistral Large':<15} {mistral_mae:<10.2f} {mistral_rmse:<10.2f} {mistral_r2:<10.4f}")
    
    # Determine winner
    if granite_mae < mistral_mae:
        winner = "IBM Granite"
        mae_advantage = mistral_mae - granite_mae
    else:
        winner = "Mistral Large"
        mae_advantage = granite_mae - mistral_mae
    
    print(f"\n🥇 WINNER: {winner}")
    print(f"MAE Advantage: {mae_advantage:.2f} points")
    print(f"Performance Improvement: {(mae_advantage/max(granite_mae, mistral_mae))*100:.1f}%")
    
    # Detailed results
    print(f"\n📋 DETAILED RESULTS:")
    print(comparison_df.to_string(index=False))
    
    # Create visualizations
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))
    
    # 1. Actual vs Predicted - IBM Granite
    ax1.scatter(comparison_df['Benchmark_Score'], comparison_df['IBM_Granite'], alpha=0.7, color='blue', s=100)
    ax1.plot([30, 90], [30, 90], '--', color='red', linewidth=2, label='Perfect Prediction')
    ax1.set_xlabel('Benchmark Score')
    ax1.set_ylabel('IBM Granite Prediction')
    ax1.set_title(f'IBM Granite: Actual vs Predicted\nMAE: {granite_mae:.2f}, R²: {granite_r2:.4f}')
    ax1.grid(True, alpha=0.3)
    ax1.legend()
    
    # 2. Actual vs Predicted - Mistral Large
    ax2.scatter(comparison_df['Benchmark_Score'], comparison_df['Mistral_Large'], alpha=0.7, color='green', s=100)
    ax2.plot([30, 90], [30, 90], '--', color='red', linewidth=2, label='Perfect Prediction')
    ax2.set_xlabel('Benchmark Score')
    ax2.set_ylabel('Mistral Large Prediction')
    ax2.set_title(f'Mistral Large: Actual vs Predicted\nMAE: {mistral_mae:.2f}, R²: {mistral_r2:.4f}')
    ax2.grid(True, alpha=0.3)
    ax2.legend()
    
    # 3. Prediction Errors
    granite_errors = comparison_df['IBM_Granite'] - comparison_df['Benchmark_Score']
    mistral_errors = comparison_df['Mistral_Large'] - comparison_df['Benchmark_Score']
    
    ax3.bar(np.arange(len(comparison_df)) - 0.2, granite_errors, 0.4, 
            label='IBM Granite', alpha=0.7, color='blue')
    ax3.bar(np.arange(len(comparison_df)) + 0.2, mistral_errors, 0.4, 
            label='Mistral Large', alpha=0.7, color='green')
    ax3.axhline(y=0, color='red', linestyle='--', alpha=0.7)
    ax3.set_xlabel('Company Index')
    ax3.set_ylabel('Prediction Error')
    ax3.set_title('Prediction Errors by Company')
    ax3.legend()
    ax3.grid(True, alpha=0.3)
    
    # 4. Performance Metrics Comparison
    metrics = ['MAE', 'RMSE']
    granite_metrics = [granite_mae, granite_rmse]
    mistral_metrics = [mistral_mae, mistral_rmse]
    
    x = np.arange(len(metrics))
    width = 0.35
    
    ax4.bar(x - width/2, granite_metrics, width, label='IBM Granite', alpha=0.7, color='blue')
    ax4.bar(x + width/2, mistral_metrics, width, label='Mistral Large', alpha=0.7, color='green')
    ax4.set_xlabel('Metrics')
    ax4.set_ylabel('Error Value')
    ax4.set_title('Model Performance Metrics')
    ax4.set_xticks(x)
    ax4.set_xticklabels(metrics)
    ax4.legend()
    ax4.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
else:
    print("❌ Cannot compare models - missing predictions")

In [ ]:
# 11. Final Summary & Insights

print("\n" + "="*80)
print("🎯 FINAL SUMMARY & INSIGHTS")
print("="*80)

print("\n📊 KEY FINDINGS:")
if 'granite_score_pred' in esg_df.columns and 'mistral_score_pred' in esg_df.columns:
    improvement_pct = ((granite_mae - mistral_mae) / granite_mae * 100) if granite_mae > mistral_mae else ((mistral_mae - granite_mae) / mistral_mae * 100)
    better_model = "Mistral Large" if mistral_mae < granite_mae else "IBM Granite"
    
    print(f"1. {better_model} outperforms with {improvement_pct:.1f}% better MAE")
    print(f"2. Best MAE achieved: {min(granite_mae, mistral_mae):.2f}")
    print(f"3. Both models show room for improvement in ESG scoring")

print(f"\n✅ METHODOLOGY VALIDATION:")
print("• Successfully followed exercise framework")
print("• Implemented proper LLM evaluation methodology")
print("• Used temperature=0 for deterministic results")
print("• Applied comprehensive error analysis")

print(f"\n🚀 RECOMMENDATIONS:")
print("• Consider fine-tuning models on ESG-specific datasets")
print("• Experiment with few-shot prompting using examples")
print("• Investigate ensemble methods combining both models")
print("• Explore retrieval-augmented generation (RAG) approaches")

print("\n" + "="*80)
print("🎉 LLM ESG ANALYSIS COMPLETE!")
print("="*80)